In [3]:
import pandas as pd

# Load the dataset to inspect its structure
file_path = '/content/drive/MyDrive/Colab Notebooks/project/Krishan Data set  finaaaaaaaaaaaal (1).xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset
data.head()


,Start Date,End Date,Event Type,Location,Venue,Availability,Weather,Day of the Week,Predicted Date,Optimized Date,Expected_Ticket_Sales,Hisrical_Ticket_Sales
0,2022-08-12,2022-08-18 00:00:00,Music,Anuradhapura,Anuradhapura Stadium,No,Cloudy,Thursday,2022-08-18,Yes,10000,6206
1,2024-11-17,2024-11-23,Fair,Jaffna,Anuradhapura Stadium,Yes,Windy,Wednesday,2024-11-20,Yes,7000,3413
2,2024-01-18,2024-01-24 00:00:00,Music,Jaffna,Galle Fort,No,Sunny,Monday,2024-01-22,Yes,8000,2537
3,2022-05-01,2022-05-07,Theater,Kandy,BMICH,Yes,Cloudy,Friday,2022-05-06,Yes,9000,6939
4,2023-07-02,2023-07-08,Theater,Kandy,Jaffna Cultural Hall,Yes,Sunny,Thursday,2023-07-06,Yes,7500,3204


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Convert date columns to datetime
data['Start Date'] = pd.to_datetime(data['Start Date'], errors='coerce')
data['End Date'] = pd.to_datetime(data['End Date'], errors='coerce')
data['Predicted Date'] = pd.to_datetime(data['Predicted Date'], errors='coerce')

# Calculate the number of days between Start and End Date
data['Date Range'] = (data['End Date'] - data['Start Date']).dt.days

# Encode categorical variables
label_encoders = {}
for col in ['Event Type', 'Location', 'Venue', 'Availability', 'Weather', 'Day of the Week ']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

# Target: Days from Start Date to Predicted Date
data['Days from Start'] = (data['Predicted Date'] - data['Start Date']).dt.days

# Features and target variable
features = ['Start Date', 'End Date', 'Event Type', 'Location', 'Venue',
            'Availability', 'Weather', 'Day of the Week ', 'Expected_Ticket_Sales', 'Date Range']
target = 'Days from Start'

# Drop rows with missing values after conversion
data_clean = data.dropna(subset=features + [target])

# Convert date features to numerical format (ordinal representation for simplicity)
data_clean['Start Date'] = data_clean['Start Date'].map(pd.Timestamp.toordinal)
data_clean['End Date'] = data_clean['End Date'].map(pd.Timestamp.toordinal)

# Split data into train and test sets
X = data_clean[features]
y = data_clean[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=40)

# Train a Random Forest model
model = RandomForestRegressor(n_estimators=250,random_state=42)
model.fit(X_train, y_train)

# Evaluate the model on the test set
score = model.score(X_test, y_test)


NameError: name 'pd' is not defined

In [3]:
# Display the column names of the dataset to identify any discrepancies
data.columns


NameError: name 'data' is not defined

In [ ]:
import pickle

# Save the trained model
with open('event_date_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the label encoders
with open('label_encoders.pkl', 'wb') as le_file:
    pickle.dump(label_encoders, le_file)


In [ ]:
import pickle

# Load the trained model
with open('event_date_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the label encoders
with open('label_encoders.pkl', 'rb') as le_file:
    label_encoders = pickle.load(le_file)


In [ ]:
def get_user_input():
    import datetime

    # Collect Start Date
    start_date_str = input("Enter Start Date (YYYY-MM-DD): ")
    start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")

    # Collect End Date
    end_date_str = input("Enter End Date (YYYY-MM-DD): ")
    end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

    # Collect Event Type
    event_type = input("Enter Event Type (e.g., Music, Fair, Theater): ")

    # Collect Location
    location = input("Enter Location (e.g., Anuradhapura, Jaffna, Kandy): ")

    # Collect Venue
    venue = input("Enter Venue (e.g., Anuradhapura Stadium, Galle Fort): ")

    # Collect Availability
    availability = input("Is the Venue Available? (Yes/No): ")

    # Collect Weather
    weather = input("Enter Weather (e.g., Cloudy, Windy, Sunny): ")

    # Collect Day of the Week
    day_of_week = input("Enter Day of the Week (e.g., Monday, Tuesday): ")

    # Collect Expected Ticket Sales
    expected_ticket_sales = float(input("Enter Expected Ticket Sales: "))

    return {
        'Start Date': start_date,
        'End Date': end_date,
        'Event Type': event_type,
        'Location': location,
        'Venue': venue,
        'Availability': availability,
        'Weather': weather,
        'Day of the Week ': day_of_week,
        'Expected_Ticket_Sales': expected_ticket_sales
    }


In [ ]:
def preprocess_input(user_input, label_encoders):
    import pandas as pd

    # Convert dates to ordinal
    user_input['Start Date'] = user_input['Start Date'].toordinal()
    user_input['End Date'] = user_input['End Date'].toordinal()

    # Calculate Date Range
    user_input['Date Range'] = user_input['End Date'] - user_input['Start Date']

    # Encode categorical variables
    for col in ['Event Type', 'Location', 'Venue', 'Availability', 'Weather', 'Day of the Week ']:
        le = label_encoders.get(col)
        if le:
            # Handle unseen labels
            if user_input[col] in le.classes_:
                user_input[col] = le.transform([user_input[col]])[0]
            else:
                # Assign a default value or handle accordingly
                user_input[col] = -1  # Example: -1 for unknown categories
        else:
            user_input[col] = -1

    # Ensure all required features are present
    features = ['Start Date', 'End Date', 'Event Type', 'Location', 'Venue',
                'Availability', 'Weather', 'Day of the Week ', 'Expected_Ticket_Sales', 'Date Range']

    input_data = [user_input.get(feature, 0) for feature in features]

    return pd.DataFrame([input_data], columns=features)


In [ ]:
def predict_event_date(user_input, model, label_encoders):
    import pandas as pd
    from datetime import datetime

    # Preprocess the input
    processed_input = preprocess_input(user_input, label_encoders)

    # Make prediction (Days from Start)
    predicted_days = model.predict(processed_input)[0]

    # Convert Days from Start to Predicted Date
    start_date_ordinal = user_input['Start Date']
    predicted_date_ordinal = start_date_ordinal + int(predicted_days)
    predicted_date = datetime.fromordinal(predicted_date_ordinal)

    return predicted_date


In [ ]:
user_input = get_user_input()

    # Make prediction
predicted_date = predict_event_date(user_input, model, label_encoders)

    # Display the result
print(f"\nPredicted Event Date: {predicted_date.strftime('%Y-%m-%d')}")


Enter Start Date (YYYY-MM-DD): 2024-10-15
Enter End Date (YYYY-MM-DD): 2024-11-15
Enter Event Type (e.g., Music, Fair, Theater): Music
Enter Location (e.g., Anuradhapura, Jaffna, Kandy): Anuradhapura
Enter Venue (e.g., Anuradhapura Stadium, Galle Fort): Anuradhapura Stadium
Is the Venue Available? (Yes/No): Yes
Enter Weather (e.g., Cloudy, Windy, Sunny): Cloudy
Enter Day of the Week (e.g., Monday, Tuesday): Monday
Enter Expected Ticket Sales: 5000

Predicted Event Date: 2024-10-18
